# bicubic interpolation 

In [77]:
import numpy as np
from scipy.interpolate import RectBivariateSpline, CubicSpline
from numba import njit, jit

# Example 2D grid (x, y) and corresponding function values z
x = np.linspace(0., 5, 50)
z = np.sin(x**2)  # Example function values
fn_ = lambda x_: np.sin(x_**2)

In [78]:
x

array([0.        , 0.10204082, 0.20408163, 0.30612245, 0.40816327,
       0.51020408, 0.6122449 , 0.71428571, 0.81632653, 0.91836735,
       1.02040816, 1.12244898, 1.2244898 , 1.32653061, 1.42857143,
       1.53061224, 1.63265306, 1.73469388, 1.83673469, 1.93877551,
       2.04081633, 2.14285714, 2.24489796, 2.34693878, 2.44897959,
       2.55102041, 2.65306122, 2.75510204, 2.85714286, 2.95918367,
       3.06122449, 3.16326531, 3.26530612, 3.36734694, 3.46938776,
       3.57142857, 3.67346939, 3.7755102 , 3.87755102, 3.97959184,
       4.08163265, 4.18367347, 4.28571429, 4.3877551 , 4.48979592,
       4.59183673, 4.69387755, 4.79591837, 4.89795918, 5.        ])

In [79]:
interp_spline = CubicSpline(x, z, extrapolate=True)

In [80]:
x_new = 0.1
interp_spline(x_new)

array(0.00999975)

In [81]:
fn_(x_new)

0.009999833334166666

In [82]:
# vectorization
# search for the nearest (but above) x and y
x_idx = np.searchsorted(x, x_new)
print(x_new)
print(x_idx)

0.1
1


In [85]:
x_idx3

1

In [87]:
# if x_idx-1==0:
x_idx1 = x_idx-1
x_idx2 = x_idx
x_idx3 = x_idx+1
x1, x2, x3 = x[x_idx1], x[x_idx2], x[x_idx3]
z1, z2, z3 = z[x_idx1], z[x_idx2], z[x_idx3]

matrixA = np.array([
    [x1**3, x1**2, x1, 1, 0, 0, 0, 0],
    [x2**3, x2**2, x2, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, x2**3, x2**2, x2, 1],
    [0, 0, 0, 0, x3**3, x3**2, x3, 1],
    [3*x2**2, 2*x2, 1, 0, -3*x2**2, -2*x2, -1, 0],
    [6*x2, 2, 0, 0, -6*x2, -2, 0, 0],
    [6*x1, 2, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 6*x3, 2, 0, 0]
])

matrixC = np.array([z1, z2, z2, z3, 0, 0, 0, 0])
coeff = np.linalg.solve(matrixA, matrixC)
matrixD = coeff[0:4]

In [88]:
matrixB = np.array([x_new**3, x_new**2, x_new, 1])
np.dot(matrixB, matrixD)

0.010001969598909926

In [84]:
# max_idx = len(x)
# if not x_idx-1==0:
x_idx1 = x_idx-2
x_idx2 = x_idx-1
x_idx3 = x_idx
x_idx4 = x_idx+1
x1, x2, x3, x4 = x[x_idx1], x[x_idx2], x[x_idx3], x[x_idx4]
z1, z2, z3, z4 = z[x_idx1], z[x_idx2], z[x_idx3], z[x_idx4]

matrixA = np.array([
    [x1**3, x1**2, x1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [x2**3, x2**2, x2, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, x2**3, x2**2, x2, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, x3**3, x3**2, x3, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, x3**3, x3**2, x3, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, x4**3, x4**2, x4, 1],
    [3*x2**2, 2*x2, 1, 0, -3*x2**2, -2*x2, -1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 3*x3**2, 2*x3, 1, 0, -3*x3**2, -2*x3, -1, 0],
    [6*x2, 2, 0, 0, -6*x2, -2, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 6*x3, 2, 0, 0, -6*x3, -2, 0, 0],
    [6*x1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 6*x4, 2, 0, 0],
])

matrixC = np.array([z1, z2, z2, z3, z3, z4, 0, 0, 0, 0, 0, 0])
coeff = np.dot(np.linalg.inv(matrixA), matrixC)
matrixD = coeff[4:8]


In [61]:
matrixB = np.array([x_new**3, x_new**2, x_new, 1])
np.dot(matrixB, matrixD)

-0.2869920012417424

In [89]:
# linear interpolation
x1, x2 = x[x_idx-1], x[x_idx]
z1, z2 = z[x_idx-1], z[x_idx]

z_new = z1 + (z2-z1)/(x2-x1)*(x_new-x1)
print(z_new)

0.010203897251716384


In [255]:
@njit
def cubic_spline_interpolator(xnew, coefficients, x):
    # Handling extrapolation
    i = np.searchsorted(x, xnew) - 1
    idx1 = xnew <= x[0]
    idx2 = xnew > x[-1]
    i[idx1] = 0
    i[idx2] = len(x) - 2

    # Calculate the relative position within the interval
    dx = xnew - x[i]

    # Calculate the interpolated value
    # Cubic polynomial: a + b*dx + c*dx^2 + d*dx^3
    a, b, c, d = coefficients[:, i]
    #result = a + b*dx + c*dx**2 + d*dx**3
    result = d + c*dx + b*dx**2 + a*dx**3
    return result